# Basic Structure of Training a Model

In [1]:
import tensorflow as tf
from datetime import datetime

## Hyper-parameters

In [2]:
num_classes = 10

epochs = 50
batch_size = 16

## Data pre-process

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

## Set up a model

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flat = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(512, activation='relu')
        self.d2 = tf.keras.layers.Dense(256, activation='relu')
        self.d3 = tf.keras.layers.Dense(128, activation='relu')
        self.d4 = tf.keras.layers.Dense(64, activation='relu')
        self.d5 = tf.keras.layers.Dense(32, activation='relu')
        self.d6 = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.flat(inputs)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)
        x = self.d5(x)
        return self.d6(x)
    
model = MyModel()

## Set callbacks
### checkpoint

In [5]:
checkpoint_path = 'checkpoints/basic_structure' + datetime.now().strftime(
    "%Y%m%d-%H%M%S") + '.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)

### tensorboard

In [6]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

### reduce learning rate

In [7]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=3,
                                                 min_lr=0.00001)

## Set optimizer, loss function, & metrics

In [8]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_acc = tf.keras.metrics.SparseCategoricalAccuracy()

## Compile the model

In [9]:
model.compile(optimizer, loss, metrics=[metric_acc])

## Fit the model

In [10]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=[checkpoint, reduce_lr, tensorboard_callback],
)

Epoch 1/50
1871/1875 [============================>.] - ETA: 0s - loss: 1.5813 - sparse_categorical_accuracy: 0.8814
Epoch 00001: val_loss improved from inf to 1.51720, saving model to checkpoints/basic_structure20200623-154955.ckpt
1875/1875 [==============================] - 8s 4ms/step - loss: 1.5812 - sparse_categorical_accuracy: 0.8814 - val_loss: 1.5172 - val_sparse_categorical_accuracy: 0.9437 - lr: 0.0010
Epoch 2/50
1872/1875 [============================>.] - ETA: 0s - loss: 1.5200 - sparse_categorical_accuracy: 0.9410
Epoch 00002: val_loss improved from 1.51720 to 1.50764, saving model to checkpoints/basic_structure20200623-154955.ckpt
1875/1875 [==============================] - 10s 5ms/step - loss: 1.5199 - sparse_categorical_accuracy: 0.9410 - val_loss: 1.5076 - val_sparse_categorical_accuracy: 0.9533 - lr: 0.0010
Epoch 3/50
1870/1875 [============================>.] - ETA: 0s - loss: 1.5104 - sparse_categorical_accuracy: 0.9505
Epoch 00003: val_loss improved from 1.50764 

Epoch 22/50
1875/1875 [==============================] - ETA: 0s - loss: 1.4795 - sparse_categorical_accuracy: 0.9816
Epoch 00022: val_loss did not improve from 1.48611
1875/1875 [==============================] - 18s 10ms/step - loss: 1.4795 - sparse_categorical_accuracy: 0.9816 - val_loss: 1.4874 - val_sparse_categorical_accuracy: 0.9736 - lr: 1.0000e-04
Epoch 23/50
1871/1875 [============================>.] - ETA: 0s - loss: 1.4793 - sparse_categorical_accuracy: 0.9818
Epoch 00023: val_loss improved from 1.48611 to 1.48585, saving model to checkpoints/basic_structure20200623-154955.ckpt
1875/1875 [==============================] - 18s 10ms/step - loss: 1.4793 - sparse_categorical_accuracy: 0.9818 - val_loss: 1.4859 - val_sparse_categorical_accuracy: 0.9752 - lr: 1.0000e-04
Epoch 24/50
1870/1875 [============================>.] - ETA: 0s - loss: 1.4790 - sparse_categorical_accuracy: 0.9821
Epoch 00024: val_loss did not improve from 1.48585
1875/1875 [==============================] -

1875/1875 [==============================] - ETA: 0s - loss: 1.4775 - sparse_categorical_accuracy: 0.9837
Epoch 00044: val_loss did not improve from 1.48573
1875/1875 [==============================] - 21s 11ms/step - loss: 1.4775 - sparse_categorical_accuracy: 0.9837 - val_loss: 1.4860 - val_sparse_categorical_accuracy: 0.9752 - lr: 1.0000e-05
Epoch 45/50
1869/1875 [============================>.] - ETA: 0s - loss: 1.4775 - sparse_categorical_accuracy: 0.9836
Epoch 00045: val_loss did not improve from 1.48573
1875/1875 [==============================] - 22s 12ms/step - loss: 1.4775 - sparse_categorical_accuracy: 0.9837 - val_loss: 1.4860 - val_sparse_categorical_accuracy: 0.9752 - lr: 1.0000e-05
Epoch 46/50
1873/1875 [============================>.] - ETA: 0s - loss: 1.4775 - sparse_categorical_accuracy: 0.9836
Epoch 00046: val_loss did not improve from 1.48573
1875/1875 [==============================] - 22s 11ms/step - loss: 1.4775 - sparse_categorical_accuracy: 0.9837 - val_loss: 1

## Check the training process via tensorboard

In [11]:
%tensorboard --logdir logs

## Load the best weights

In [12]:
model.load_weights(checkpoint_path)

## Evaluate the model

In [ ]:
model.evaluate(x_test, y_test)

  1/313 [..............................] - ETA: 0s - loss: 1.4612 - sparse_categorical_accuracy: 1.0000